Chapter 07

# 初等行变换求解线性方程组
《线性代数》 | 鸢尾花书：数学不难

该代码实现了高斯消元法（Gaussian Elimination）来求解线性方程组 $Ax = b$，其中 $A$ 是系数矩阵，$b$ 是右端项向量，$x$ 是待求解向量。高斯消元的核心思想是通过行变换将增广矩阵化为上三角矩阵，然后通过回代（Back Substitution）求解未知数。下面从数学角度详细描述代码执行的数学过程。

---

### **1. 形成增广矩阵**
首先，将线性方程组 $Ax = b$ 组装成增广矩阵：
$$
[A | b] =
\begin{bmatrix}
a_{11} & a_{12} & \cdots & a_{1n} & | b_1 \\
a_{21} & a_{22} & \cdots & a_{2n} & | b_2 \\
\vdots & \vdots & \ddots & \vdots & | \vdots \\
a_{n1} & a_{n2} & \cdots & a_{nn} & | b_n
\end{bmatrix}
$$
其中，$A$ 是 $n \times n$ 的系数矩阵，$b$ 是 $n \times 1$ 的列向量。代码通过 `np.hstack([A, b.reshape(-1, 1)])` 来构造这个增广矩阵。

---

### **2. 高斯消元**
高斯消元的目标是通过一系列初等行变换，将增广矩阵转换为上三角矩阵：
$$
\begin{bmatrix}
1 & * & * & \cdots & * & | * \\
0 & 1 & * & \cdots & * & | * \\
0 & 0 & 1 & \cdots & * & | * \\
\vdots & \vdots & \vdots & \ddots & \vdots & \vdots \\
0 & 0 & 0 & \cdots & 1 & | *
\end{bmatrix}
$$
在此过程中，主要涉及以下步骤：

#### **(a) 选取主元并交换行**
在第 $i$ 步时，代码使用 `np.argmax(np.abs(Ab[i:, i])) + i` 选择当前列中绝对值最大的元素作为主元，并交换当前行 $i$ 和主元所在行 `max_row`：
$$
\text{若 } |a_{pi}| = \max\{|a_{ji}|, j \geq i\}, \text{则交换 } i \text{ 行和 } p \text{ 行}
$$
这一操作可以减少数值误差，提高计算稳定性。

#### **(b) 归一化主元**
将当前主元归一化为 1，即对当前行的所有元素进行除法运算：
$$
\frac{A[i, j]}{A[i, i]}, \quad \forall j \geq i
$$
代码执行：
```python
Ab[i] = Ab[i] / Ab[i, i]
```

#### **(c) 消去主元下方元素**
对所有 $j > i$ 的行执行消元：
$$
\text{令 } m_{ji} = \frac{A[j,i]}{A[i,i]}, \quad A[j, k] = A[j, k] - m_{ji} A[i, k] \quad \forall k \geq i
$$
等价于用第 $i$ 行的倍数消去第 $i$ 列下方的元素：
```python
Ab[j] = Ab[j] - Ab[j, i] * Ab[i]
```
最终，增广矩阵被化为上三角矩阵：
$$
\begin{bmatrix}
1 & a_{12}' & a_{13}' & \cdots & a_{1n}' & | b_1' \\
0 & 1 & a_{23}' & \cdots & a_{2n}' & | b_2' \\
0 & 0 & 1 & \cdots & a_{3n}' & | b_3' \\
\vdots & \vdots & \vdots & \ddots & \vdots & \vdots \\
0 & 0 & 0 & \cdots & 1 & | b_n'
\end{bmatrix}
$$

---

### **3. 回代求解**
已得到上三角矩阵，接下来使用回代法计算 $x$：
对于最后一行：
$$
x_n = b_n'
$$
对前面的行，逐步计算：
$$
x_i = b_i' - \sum_{j=i+1}^{n} a_{ij}' x_j, \quad \forall i = n-1, n-2, \dots, 1
$$
代码执行：
```python
x[i] = Ab[i, -1] - np.dot(Ab[i, i+1:n], x[i+1:n])
```
最终得到解向量：
$$
x = \begin{bmatrix} x_1 \\ x_2 \\ \vdots \\ x_n \end{bmatrix}
$$

---

### **4. 示例计算**
设：
$$
A = \begin{bmatrix} 1 & 3 & 6 \\ 2 & 7 & 14 \\ 0 & 2 & 5 \end{bmatrix}, \quad
b = \begin{bmatrix} 25 \\ 58 \\ 19 \end{bmatrix}
$$
执行高斯消元后得到：
$$
\begin{bmatrix} 1 & 3.5 & 7 & | 29 \\ 0 & 1 & 2.5 & | 9.5 \\ 0 & 0 & 1 & | 3 \end{bmatrix}
$$
回代计算：
$$
x_3 = 3, \quad x_2 = 9.5 - 2.5 \times 3 = 2, \quad x_1 = 29 - 3.5 \times 2 - 7 \times 3 = 1
$$
最终解为：
$$
x = \begin{bmatrix} 1 \\ 2 \\ 3 \end{bmatrix}
$$

---

### **5. 结论**
代码实现了高斯消元法，先通过选取主元、行交换、归一化和消元操作，将增广矩阵转化为上三角形式，再使用回代求解线性方程组。该方法适用于求解具有唯一解的方程组，且数值稳定性较好。

## 初始化

In [5]:
import numpy as np

## 自定义函数

In [7]:
def gauss_elimination(A, b):
    # 确保使用浮点数计算
    A = A.astype(float)  
    b = b.astype(float)
    n = len(b)

    # 构造增广矩阵 [A|b]
    Ab = np.hstack([A, b.reshape(-1, 1)])

    # 原始矩阵A > 上三角
    for i in range(n):
        # 1) 行交换，选择绝对值最大为主元
        # 主元：主对角线元素
        max_row = np.argmax(np.abs(Ab[i:, i])) + i
        Ab[[i, max_row]] = Ab[[max_row, i]]  
        # 交换行, 行 i <> 行 max_row

        # 2) 行数乘，归一化主元，主元变为 1
        Ab[i] = Ab[i] / Ab[i, i]

        # 3) 行倍加，消元，使主元下面所有行的 i列变为 0
        for j in range(i+1, n):
            # Ab[j] -= Ab[j, i] * Ab[i]
            Ab[j] = Ab[j] - Ab[j, i] * Ab[i]

    # 回代求解，上三角矩阵 > 单位矩阵
    x = np.zeros(n)
    for i in range(n-1, -1, -1):
        x[i] = Ab[i, -1] - np.dot(Ab[i, i+1:n], x[i+1:n])

    return x

## 定义矩阵 A 和向量 b

In [9]:
A = np.array([[1, 3, 6], 
              [2, 7, 14],
              [0, 2, 5]])
b = np.array([25, 58, 19])

## 求解 Ax = b

In [11]:
x = gauss_elimination(A, b)
x

array([1., 2., 3.])

作者	**生姜DrGinger**  
脚本	**生姜DrGinger**  
视频	**崔崔CuiCui**  
开源资源	[**GitHub**](https://github.com/Visualize-ML)  
平台	[**油管**](https://www.youtube.com/@DrGinger_Jiang)		
		[**iris小课堂**](https://space.bilibili.com/3546865719052873)		
		[**生姜DrGinger**](https://space.bilibili.com/513194466)  